In [1]:
# Stealing stuff... http://brandonrose.org/clustering
import numpy as np
import pandas as pd
import nltk, re, itertools
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.metrics import *
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.snowball import SnowballStemmer
from datetime import datetime, timedelta, date
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold
from collections import defaultdict

from data_science_toolkit.data_visualization import get_fig_ax, visualize_class_distribution, top_n_tokens_plot_from_counter

# Include a paragraph in your final report about why you changed your topic
- Predict more or less than avg number of retweets for that year for each tweet
- LDA for topic modelling (investigate)
- Apriori for combos of words
- Binary classifier
- In general we use clustering or topic modelling to help understand the data, but we don't usually use it for prediction

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')

In [3]:
output_col = 'above_monthly_avg'
rand_state = 1

In [4]:
%matplotlib inline
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

In [5]:
stock_tweet = pd.read_csv('./since_election_with_cluster_trump_tweets_sp500.csv')
stock_tweet['created_at']= pd.to_datetime(stock_tweet['created_at']) 
stock_tweet['dow'] = stock_tweet.dow.astype('category')
stock_tweet['num_links'] = stock_tweet.num_links.astype('category')
# stock_tweet['num_words'] = stock_tweet.num_words.astype('category')
# stock_tweet['cluster_number'] = stock_tweet.cluster_number.astype('category')
stock_tweet['created_hour'] = stock_tweet.created_hour.astype('category')

In [6]:
for col in [x for x in stock_tweet.columns.tolist() if '_apr_' in x]:
    stock_tweet[col] = stock_tweet[col].astype('category')

In [7]:
stock_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9349 entries, 0 to 9348
Data columns (total 27 columns):
text                           9349 non-null object
created_at                     9349 non-null datetime64[ns]
retweet_count                  9349 non-null float64
favorite_count                 9349 non-null int64
is_retweet                     9349 non-null bool
after_dir                      9349 non-null int64
1_open_dir                     9349 non-null int64
1_close_dir                    9349 non-null int64
2_open_dir                     9349 non-null int64
2_close_dir                    9349 non-null int64
eow_close_dir                  9349 non-null int64
sow_open_dir                   9349 non-null int64
sig_up                         9349 non-null int64
sig_down                       9349 non-null int64
is_sig                         9349 non-null int64
preprocessed_text              9349 non-null object
created_hour                   9349 non-null category
dow        

In [8]:
stop_words = set(stopwords.words('english')).union({''})

In [9]:
len(stock_tweet.index)

9349

In [10]:
from data_science_toolkit.dataset_ops import classifier_train_val_test_dfs
# Need to shuffle so our train/test dates are randomized!!
train_df, val_df, test_df = classifier_train_val_test_dfs(stock_tweet.sample(frac=1, random_state=rand_state), output_col, 
                                                          train_val_test_split=(0.8,0.1,0.1), random_state=rand_state)
train_df = train_df.append(val_df)

9349it [00:00, 610027.20it/s]
8414it [00:00, 704450.84it/s]


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8414 entries, 2038 to 2895
Data columns (total 27 columns):
text                           8414 non-null object
created_at                     8414 non-null datetime64[ns]
retweet_count                  8414 non-null float64
favorite_count                 8414 non-null int64
is_retweet                     8414 non-null bool
after_dir                      8414 non-null int64
1_open_dir                     8414 non-null int64
1_close_dir                    8414 non-null int64
2_open_dir                     8414 non-null int64
2_close_dir                    8414 non-null int64
eow_close_dir                  8414 non-null int64
sow_open_dir                   8414 non-null int64
sig_up                         8414 non-null int64
sig_down                       8414 non-null int64
is_sig                         8414 non-null int64
preprocessed_text              8414 non-null object
created_hour                   8414 non-null category
dow     

In [12]:
# train_df = train_df[train_df[output_col].sample()
min_class_count = min(len(train_df[train_df[output_col] == 0].index), len(train_df[train_df[output_col] == 1].index))
train_df = train_df[train_df[output_col] == 0].sample(min_class_count, random_state=rand_state).append(train_df[train_df[output_col] == 1].sample(min_class_count, random_state=rand_state))
train_df = train_df.sample(frac=1, random_state=rand_state)

In [13]:
print(len(train_df[train_df[output_col] == 0].index), len(train_df[train_df[output_col] == 1].index))

3464 3464


In [14]:
train_in = train_df['preprocessed_text'].tolist()
train_out = train_df[output_col].tolist()
val_in = val_df['preprocessed_text'].tolist()
val_out = val_df[output_col].tolist()
test_in = test_df['preprocessed_text'].tolist()
test_out = test_df[output_col].tolist()

In [15]:
len(val_out)

842

In [16]:
len(test_out)

935

# STOLE A BUNCH OF SHIT FROM HERE: https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0

In [17]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def already_preprocessed_tokenize(text):
    tokens = text.split(" ")
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def dank_tokenize(text):
    return text.split(" ")

In [18]:
stemmer = SnowballStemmer("english")

In [19]:
preprocessed_vocab = ' '.join(stock_tweet['preprocessed_text'].tolist())
total_vocab = list(set(tokenize_only(preprocessed_vocab)))
total_vocab_stemmed = [stemmer.stem(t) for t in total_vocab]
vocab_frame = pd.DataFrame({'words': total_vocab}, index = total_vocab_stemmed)
print('there are ', str(vocab_frame.shape[0]), ' items in vocab_frame')

there are  13552  items in vocab_frame


In [20]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
# We create the preprocessing pipelines for both numeric and categorical data.
numeric_features = ['num_words']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['created_hour', 'dow', 'num_links'] + [x for x in stock_tweet.columns.tolist() if '_apr_' in x]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='error', categories='auto'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
super_ting = Pipeline([
    ('features', FeatureUnion([
        ('preprocessor', preprocessor),
        ('text', Pipeline([
            ('colext', TextSelector('preprocessed_text')),
            ('tfidf', TfidfVectorizer(max_df=0.9,min_df=3,use_idf=True, tokenizer=already_preprocessed_tokenize, ngram_range=(1,4)))
        ])),
        ('bow', Pipeline([
            ('colext', TextSelector('preprocessed_text')),
            ('bow', CountVectorizer(max_df=0.5, min_df=5, tokenizer=dank_tokenize, ngram_range=(1, 5)))
        ])),
    ]))
])
clf = Pipeline(steps=[('yeet', super_ting),
                      ('classifier', SVC(probability=True, gamma='scale', kernel='linear'))])
svm_rbf = Pipeline(steps=[('yeet', super_ting),
                      ('classifier', SVC(probability=True, gamma='scale', kernel='rbf'))])
random_forest = Pipeline(steps=[('yeet', super_ting),
                        ('classifier', RandomForestClassifier(n_estimators=100, random_state=rand_state))])

In [30]:
model_1 = defaultdict(list)
model_2 = defaultdict(list)
model_avg = defaultdict(list)
kf = KFold(n_splits=3, random_state=rand_state)
for train_index, test_index in kf.split(train_df):
    print("New Fold...")
    curr_train_df = train_df.iloc[train_index]
    curr_train_out = curr_train_df[output_col].tolist()
    curr_test_df = train_df.iloc[test_index]
    curr_test_out = curr_test_df[output_col].tolist()
    random_forest.fit(curr_train_df, curr_train_out)
    rf_out = random_forest.predict_proba(curr_test_df)
    model_2['roc'].append(roc_auc_score(curr_test_out, rf_out[:,1]))
    model_2['acc'].append(accuracy_score(curr_test_out, np.round(rf_out[:,1])))
    svm_rbf.fit(curr_train_df, curr_train_out)
    svm_rbf_out = svm_rbf.predict_proba(curr_test_df)
    model_1['roc'].append(roc_auc_score(curr_test_out, svm_rbf_out[:,1]))
    model_1['acc'].append(accuracy_score(curr_test_out, np.round(svm_rbf_out[:,1])))
    averaged_preds = np.mean(np.array([rf_out[:,1], svm_rbf_out[:,1]]), axis=0)
    model_avg['roc'].append(roc_auc_score(curr_test_out, averaged_preds))
    model_avg['acc'].append(accuracy_score(curr_test_out, np.round(averaged_preds)))
    print(model_1)
    print(model_2)
    print(model_avg)

New Fold...
defaultdict(<class 'list'>, {'acc': [0.693939393939394], 'roc': [0.7509849906543038]})
defaultdict(<class 'list'>, {'acc': [0.6662337662337663], 'roc': [0.72797350965572]})
defaultdict(<class 'list'>, {'acc': [0.6835497835497836], 'roc': [0.7485820258230623]})
New Fold...
defaultdict(<class 'list'>, {'acc': [0.693939393939394, 0.6990038977912516], 'roc': [0.7509849906543038, 0.7715522610252462]})
defaultdict(<class 'list'>, {'acc': [0.6662337662337663, 0.6886097877869207], 'roc': [0.72797350965572, 0.7498934505077608]})
defaultdict(<class 'list'>, {'acc': [0.6835497835497836, 0.7067994802944998], 'roc': [0.7485820258230623, 0.7705021482478611]})
New Fold...


KeyboardInterrupt: 

In [23]:
for model in [model_1, model_2, model_avg]:
    print("Acc: {}, ROC: {}".format(np.mean(model['acc']), np.mean(model['roc'])))

Acc: 0.6950059276174478, ROC: 0.7582402417916453
Acc: 0.6750878831000096, ROC: 0.7359917278930913
Acc: 0.6916871492878421, ROC: 0.7561529286211748


In [24]:
# train_df.head(1)

In [25]:
len(train_out) == len(train_df.index)

True

In [26]:
# clf.fit(train_df, train_out)
# lr = clf.predict_proba(test_df)

In [27]:
# print('ROC-AUC yields ' + str(roc_auc_score(test_out, lr[:,1])))
# print("Accuracy: ", accuracy_score(test_out, np.round(lr[:,1])))

In [28]:
accuracy_score(test_out, np.round(averaged_preds))

ValueError: Found input variables with inconsistent numbers of samples: [935, 2309]

In [ ]:
roc_auc_score(test_out, averaged_preds)

In [ ]:
lr_precision, lr_recall, _ = precision_recall_curve(test_out, averaged_preds)
# lr_prec, lr_rec, _ = precision_recall_curve(test_out, lr[:,1])
lr2_prec, lr2_rec, _ = precision_recall_curve(test_out, lr2[:,1])
lr3_prec, lr3_rec, _ = precision_recall_curve(test_out, lr3[:,1])
plt.plot(lr_recall, lr_precision, marker='.', label='Averaged')
# plt.plot(lr_rec, lr_prec, marker='.', label='lr')
plt.plot(lr2_rec, lr2_prec, marker='.', label='lr2')
plt.plot(lr3_rec, lr3_prec, marker='.', label='lr3')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
test_df['pred'] = list(np.round(averaged_preds))
test_df['is_fp'] = np.where(((test_df["1_open_dir"] == 0) & (test_df['1_open_dir'] != test_df['pred'])), 1, 0)
test_df['is_fn'] = np.where(((test_df["1_open_dir"] == 1) & (test_df['1_open_dir'] != test_df['pred'])), 1, 0)

In [ ]:
test_df[test_df.is_fp == 1][['pred', '1_open_dir','preprocessed_text']]